In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/projects/ashehu/akabir4/venvs/hopper_transformers_editable/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("Rostlab/prot_t5_xl_uniref50")
model = model.to("cpu")

In [8]:
model

T5ForConditionalGeneration(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dro

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", use_fast=False, force_download=True) # this did not work
tokenizer

T5Tokenizer(name_or_path='Rostlab/prot_t5_xl_uniref50', vocab_size=128, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '

In [4]:
# from transformers import T5Tokenizer
# tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50")
# tokenizer

In [39]:
import re
seq = "ASDX"
seq = re.sub(r"[UZOB]", "X", seq) # replacing unknown amino acids 
seq = list(seq)
seq[0] = "<extra_id_0>" # mut_pos should be 0-indexed. replace AA by special mask token used by T5
seq = " ".join(seq)
input_ids = tokenizer(seq, return_tensors="pt").input_ids.to("cpu")
input_ids

tensor([[127,   7,  10,  23,   1]])

In [40]:
import torch
with torch.no_grad():
    logits = model(input_ids, labels=input_ids).logits
logits = logits.squeeze().detach().numpy() 
logits

array([[-8.9623642e+00, -1.1519006e+01, -2.6319691e+01, -2.1865085e-01,
         2.9277468e-01, -1.0201454e-02, -2.7182025e-01,  5.2822196e-01,
        -2.8677055e-01,  1.6395064e-01,  2.4561551e-01, -3.6883721e-01,
        -3.4541780e-01, -3.1883174e-01, -2.5528312e-01, -5.1485687e-01,
        -7.5694716e-01, -5.3059280e-01, -1.0208641e+00, -1.0341456e+00,
        -1.2978854e+00, -1.5801252e+00, -1.6105332e+00, -9.6087420e-01,
        -2.6679211e+01, -2.6762032e+01, -2.6532936e+01, -2.6364166e+01,
        -2.5991905e+01, -2.6679920e+01, -2.6217539e+01, -2.6640343e+01,
        -2.6261005e+01, -2.6664814e+01, -2.6575035e+01, -2.6799864e+01,
        -2.6431791e+01, -2.6238785e+01, -2.6557615e+01, -2.6683323e+01,
        -2.5811712e+01, -2.6584278e+01, -2.6596745e+01, -2.6556015e+01,
        -2.5744869e+01, -2.6089424e+01, -2.6392120e+01, -2.5533455e+01,
        -2.6202869e+01, -2.6128756e+01, -2.6233582e+01, -2.6652227e+01,
        -2.6194559e+01, -2.5723972e+01, -2.6535376e+01, -2.62379

In [14]:
import re
seq = "ASDX"

seq = re.sub(r"[UZOB]", "X", seq) # replacing unknown amino acid with unknown token
seq = list(seq)

mut_pos_zero_idxed = 2 # the outputs of mutant A_DX and AS_X are different at every positions.
seq[mut_pos_zero_idxed] = '<extra_id_0>'# tokenizer.mask_token #'<extra_id_0>' # mut_pos must be 0-indexed. replace AA by special mask token used by the model

seq = " ".join(list(seq)) # space separated amino acids
print(seq)

# <eos> token at the end
# starts from 0-index
input_ids = tokenizer.batch_encode_plus(
            [seq], add_special_tokens=True, padding="longest"
        )
print(input_ids)

print(tokenizer.convert_ids_to_tokens(3), tokenizer.convert_tokens_to_ids('▁A'))
print(tokenizer.convert_ids_to_tokens(127), tokenizer.convert_tokens_to_ids('<extra_id_0>'))

import torch
tokenized_sequences = torch.tensor(input_ids["input_ids"]).to("cpu")
attention_mask = torch.tensor(input_ids["attention_mask"]).to("cpu")

A S <extra_id_0> X
{'input_ids': [[3, 7, 127, 23, 1]], 'attention_mask': [[1, 1, 1, 1, 1]]}
▁A 3
<extra_id_0> 127


In [15]:
with torch.no_grad():
    logits = model(input_ids=tokenized_sequences, attention_mask=attention_mask, decoder_input_ids=tokenized_sequences).logits

logits = logits.squeeze().cpu().numpy()
logits = logits[0:4]
print(logits)
print(logits.shape)

[[-17.390213   -9.293228  -50.622787   -7.236208  -10.624226   -8.6953535
  -10.065987   -9.033568   -9.414228   -9.940994  -10.468842  -10.259581
  -12.276777   -9.225728  -11.641765  -12.024082  -11.323845  -12.420128
  -12.841398  -12.613513  -11.722965  -12.178925  -11.511997   -4.7033224
  -50.698647  -51.0035    -50.929955  -50.378654  -49.78744   -51.267982
  -50.573204  -51.000893  -50.63575   -51.379814  -50.905254  -51.337036
  -50.694023  -50.559666  -51.067024  -51.027008  -49.835648  -50.958687
  -51.259655  -51.352947  -50.01358   -50.387268  -50.794064  -49.550823
  -50.35047   -50.089256  -50.655922  -51.105324  -50.492165  -49.899284
  -51.121902  -50.767487  -49.83354   -49.934505  -50.972477  -50.685616
  -51.007286  -50.040413  -51.130943  -50.581734  -49.522335  -50.701363
  -49.25947   -49.57488   -50.061813  -51.410515  -51.1847    -51.082794
  -51.39098   -50.893303  -50.307026  -49.09309   -50.58528   -51.06971
  -50.63497   -51.109264  -50.554596  -50.864098  